#### Legal Notice
<span style="font-family: 'Monospace'; font-size: 0.6em;">
The BHC Complexity Toolkit is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.
<br/>
The BHC Complexity Toolkit is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.
<br/>
You should have received a copy of the GNU General Public License along with the BHC Complexity Toolkit.  If not, see <https://www.gnu.org/licenses/>.
<br/><br/>
Copyright 2019, Mark D. Flood
<br/>
Author: Mark D. Flood
<br/>
Last revision: 22-Jun-2019
</span>

# Bank Holding Company (BHC) organizational complexity 
This notebook presents Python source code implementing a method for analyzing the topological complexity of BHC ownership and control hierarchies. Each BHC hierarchy has the structure of a [directed graph](https://en.wikipedia.org/wiki/Directed_graph), where: 

 *   _Nodes_ represent BHC legal entities and their subsidiary firms, and 
 *   _Edges_ represent ownership/control relationships among the nodes

Each edge is directed _from_ the controlling firm (the "parent") _to_ the subsidiary ("offspring") firm. 

## Data
The _Federal Reserve_ collects, via its form FR Y-10, the input data required to implement this methodology:

 *   Form FR Y-10: [**Report of Changes in Organizational Structure**](https://www.federalreserve.gov/apps/reportforms/reportdetail.aspx?sOoYJ+5BzDaGhRRQo6EFJQ==)
 
     _This report provides data on organizational structural changes for the reportable companies listed in the respondent panel section below. There are eight schedules: Banking; Savings and Loan; Nonbanking; Merger; 4(k); Domestic Branch; Foreign Branches of U.S. Banking Organizations; and Branch, Agency, and Representative Office._
     
The data form the core of the _Federal Reserve's_ National Information Center (NIC) database. The interagency _Federal Financial Institutions Examination Council_ (FFIEC) makes the NIC data available via their [**NIC Public Website** (NPW)](https://www.ffiec.gov/npw):

 *   NIC National Information Center: [**Data Download**](https://www.ffiec.gov/npw/FinancialReport/DataDownload)
 
     _Tables of structure information for select banks and institutions for which the Federal Reserve has a supervisory, regulatory, or research interest_

The Python code assumes you have downloaded the (zipped) XML version of five files:

 *   **Attributes - Active**
 *   **Attributes - Branches**
 *   **Attributes - Closed**
 *   **Relationships**
 *   **Transformations**

You should download all five files to capture an internally consistent snapshot of the data at a point in time. The FFIEC updates the data at least quarterly. 

The FFIEC now ([since 2018](https://www.ffiec.gov/npw/Home/About)) also provides these files in comma-separated-value (CSV) format; the earlier version of the NPW provided XML only. If you choose to download the CSV files instead of XML, then you should skip Step 2 below. 

The data dictionary describing the NIC dataset (version 2.0, dated July 2018) is available here:

 *   [**Bulk Data Download - Data Dictionary and Reference Guide**](https://www.ffiec.gov/npw/StaticData/DataDownload/NPW%20Data%20Dictionary.pdf)
 
     _The Bulk Data Download feature was developed in response to growing demand from the public for data in bulk format. Data being provided are considered non-confidential, public data. As our first iteration of this feature, we are releasing the tables related to attributes, relationships,and transformations. Details of these tables can be found in the subsequent sections of this Data Dictionary._
     
## Mathematical details of complexity measurement
The formal methodology implemented by the Python modules is described in detail in the following paper:

 *   M. Flood, D. Kenett, R. Lumsdaine, and J. Simon (2017), "The Complexity of Bank Holding Companies: A Topological Approach," [Working Paper 23755](http://www.nber.org/papers/w23755), _National Bureau of Economic Research_, August. 
 
     _Large bank holding companies (BHCs) are structured into intricate ownership hierarchies involving hundreds or even thousands of legal entities. Each subsidiary in these hierarchies has its own legal form, assets, liabilities, managerial goals, and supervisory authorities. In the event of BHC default or insolvency, regulators may need to resolve the BHC and its constituent entities. Each entity individually will require some mix of cash infusion, outside purchase, consolidation with other subsidiaries, legal guarantees, and outright dissolution. The subsidiaries are not resolved in isolation, of course, but in the context of resolving the consolidated BHC at the top of the hierarchy. The number, diversity, and distribution of subsidiaries within the hierarchy can therefore significantly ease or complicate the resolution process. We propose a set of related metrics intended to assess the complexity of the BHC ownership graph. These proposed metrics focus on the graph quotient relative to certain well identified partitions on the set of subsidiaries, such as charter type and regulatory jurisdiction. The intended measures are mathematically grounded, intuitively sensible, and easy to implement. We illustrate the process with a case study of one large U.S. BHC._
     
An alternate version of this paper is available from the Office of Financial Research:

 *   M. Flood, D. Kenett, R. Lumsdaine, and J. Simon (2017), "The Complexity of Bank Holding Companies: A New Measurement Approach," [OFR Working Paper 17-03](https://www.financialresearch.gov/working-papers/2017/09/29/complexity-of-bank-holding-companies/), _Office of Financial Research_, September. 

## Software

### Setup

The software consists of a set of Python modules, designed to run in the following sequence:

 1.   \[**zip2xml**\] Unpacking the _National Information Center_ (NIC) data to reveal the raw XML
 1.   \[**xml2csv**\] Parsing the XML files to create CSV versions of the same data
 1.   \[**csv2sys**\] Assembling a network (graph) representation of the banking system
 1.   \[**sys2bhc**\] Extracting individual BHC subgraphs from the system
 1.   \[**bhc2out**\] Calculating a range of complexity metrics on individual BHC graphs
 
For example, the module **zip2xml.py** contains the code needed to unpack the NIC downloads (a set of \*.zip files) into the underlying \*.xml files. 

This sequence is encoded in the notebook cells below, with each cell invoking one of the Python modules indicated in square brackets above. You can execute the modules in any of several alternative modes:

 *   Through the steps (cells) in this notebook
 *   By invoking the modules from the command line
 *   By running the modules in an interactive Python shell
 
The mode of execution should not affect the outcome of the calculations. 

 *   [Python 3](https://www.python.org/downloads/)
 *   [Installing Python](https://docs.python-guide.org/starting/installation/)

### Dependencies
The software requires Python version 3.x to run correctly. 

In addition, the sofware uses the following supplementary modules:

 *   tqdm -- v4.0 or higher
 *   networkx -- v2.2 or higher
 *   numpy -- v1.15 or higher
 *   pandas -- v0.22 or higher



## Execution sequence

### Step 0 - Configuration
You can control the execution process through a collection of configuration parameters. These parameters reside in the **bhc_complex.ini** configuration file, as key-value pairs. The availability of configuration parameters should discourage tinkering with the (tested) Python code. 

The configuration file is divided into _sections_, indicated by labels in square brackets. For example, this snippet shows the beginning of the **[DEFAULT]** section in the configuration file:

   ***
```python
# =============================================================================
# ====== Default configuration parameters, for all modules ====================
# =============================================================================
[DEFAULT]

# Default verbosity level
verbose=True

# Default high verbosity level
veryverbose=False
```
   ***

All parameter values in the **bhc_complex.ini** file are treated as strings. For simple (one-line) parameter settings, no quotes are required. For example, in the preceding snippet, the **verbose** parameter is assigned the string value, '_True_'. The Python program converts this string into the equivalent boolean value at runtime. 

Lists and dictionaries in the **bhc_complex.ini** file are handled slightly differently. First, these parameter values are typically read from multiple consecutive lines in the configuration file. The closing brace indicating the end of the list or dictionary must be indented by at least one character (if the closing brace is not indented, it will be treated as the start of a new parameter key). The resulting string is evaluated as standard Python. So, string values _within_ lists and dictionaries should be enclosed in quotes. For example, in the following snippet, the **bhclist** parameter is a list of two integers (ID_RSSDs), while **asoflist** is a list of four strings (note the single quotes indicating string values). 

   ***
```python
# =============================================================================
# ========= Configuration parameters for sys2bhc.py ===========================
# =============================================================================
[sys2bhc]

bhclist=[
    1073551, # Wachovia
    1120754, # Wells Fargo
 ]
 
asoflist=[
    '2006Q4', 
    '2008Q3', 
    '2008Q4', 
    '2010Q4', 
 ]
```
   ***

You can also override the parameter values in the **bhc_complex.ini** file at runtime by modifying the configuration after you have read it from disk. The configuration indexes the parameters first by _section_ and then by _key_. For example, to tweak the **indir** directory location, you might submit the following code: 

   ***
```python
import bhc_datautil as UTIL
CONFIG = UTIL.read_bhc_config()
CONFIG['zip2xml']['indir']='../../data/2016redo'
```
   ***
Be sure to enclose all (simple) parameter values within quotes when overriding. 

In [9]:
# STEP 0 - Configuration
#
# The ability to reload modules is useful
from importlib import reload

# Add the path to Python code for BHC complexity
import sys
sys.path.append('./bhccpx/bhccpx/')

# Base configuration is stored as key-value pairs in the bhc_config.ini file
import bhc_datautil as UTIL
CONFIG = UTIL.read_config()

# Last-minute overrides choices within CONFIG:
CONFIG['DEFAULT']['datadir']='./data/2016redo'
CONFIG['DEFAULT']['cachedir']='./data/2016redo/cache'

CONFIG['www2dat']['nic_dir']='./data/NIC'
CONFIG['www2dat']['nic_subdir']='2016redo'

### Step 1 - Unpacking the NIC data downloads
This step unzips the files downloaded from the FFIEC's NIC website. These can be either the XML or CSV downloads. 

Note that the FFIEC has not adhered to a fixed naming convention for filenames over time. The names of the downloaded ZIP files can vary, depending on the vintage of the download.  

You must run **_Step 0_** first, to load the configuration, before running this step. 

The parameters for this step cover just the source location where the **\*.zip** files reside, the target location for storing their unpacked contents, and the names of the five **\*.zip** files themselves.  


In [10]:
# STEP 1 - Unpacking the NIC data downloads
#
# Unzips NIC downloads to expose XML or CSV files
import zip2xml as z2x
# Reload the module, just to ensure the notebook kernel has the latest version from disk
reload(z2x)

# Some configuration overrides:
CONFIG['zip2xml']['nic_format']='XML'
CONFIG['zip2xml']['nic_xmltgt_attributesactive']='20160630_ATTRIBUTES_ACTIVE.zip'
CONFIG['zip2xml']['nic_xmltgt_attributesbranch']='20160630_ATTRIBUTES_BRANCH.zip'
CONFIG['zip2xml']['nic_xmltgt_attributesclosed']='20160630_ATTRIBUTES_INACTIVE.zip'
CONFIG['zip2xml']['nic_xmltgt_relationships']='20160630_RELATIONSHIPS.zip'
CONFIG['zip2xml']['nic_xmltgt_transformations']='20160630_TRANSFORMATIONS.zip'

# Now, do the actual work:
z2x.unzip_data(CONFIG)

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


### Step 2 (optional) - Converting the NIC XML files to CSV
This step parses the XML downloads to create equivalent CSV files. If you downloaded the CSV versions of the NIC data files from the FFIEC website, you should skip this step. (Prior to 2008, the FFIEC provided the public data downloads only as XML files.)

Note that the FFIEC has not adhered to a fixed naming convention for filenames over time. The names of the XML files unpacked in **_Step 1_** can vary, depending on the vintage of the download.  

You must run **_Step 0_** first, to load the configuration, before running this step. 

The parameters for this step cover the source location where the **\*.xml** files reside, the target location for storing their unpacked contents, and the names of the five **\*.xml** files themselves. The parameters also include templates for organizing the columns of the CSV output in your preferred order. 

In [11]:
# STEP 2 (optional) - Converting the NIC XML files to CSV
#
# Unzips NIC downloads to expose XML or CSV files
import xml2csv as x2c
# Reload the module, just to ensure the notebook kernel has the latest version from disk
reload(x2c)

# Some configuration overrides:
CONFIG['xml2csv']['attributesactive']='20160630_ATTRIBUTES_ACTIVE.XML'
CONFIG['xml2csv']['attributesbranch']='20160630_ATTRIBUTES_BRANCH.XML'
CONFIG['xml2csv']['attributesclosed']='20160630_ATTRIBUTES_INACTIVE.XML'
CONFIG['xml2csv']['relationships']='20160630_RELATIONSHIPS.XML'
CONFIG['xml2csv']['transformations']='20160630_TRANSFORMATIONS.XML'

# Now, do the actual work:
x2c.parse_nic(CONFIG)

### Step 3 - Build banking system network snapshots
This step creates network objects representing quarterly snapshots of the banking system. Each quarterly network derives only from the NIC **Relationships** table, so they do not (at this stage) incorporate the attributes describing each node in the network. 

Two parameters, **asofdate0** and **asofdate1** control the start and end of the quarterly sample, respectively. These parameters are strings of the form _yyyyQn_, where _yyyy_ represents the year and _n_ represents the calendar quarter (1-4).  

The network objects are pickled and cached in the output directory, **outdir**. To speed performance, this step checks whether a cached **\*.pik** file exists before creating a new network object. To override this behavior, ensure that the **clearcache** parameter is set to _True_; this will force the deletion (and recreation) of cached network object for the selected quarters. 

The **parallel** parameter toggles whether to exploit multiple CPU cores to speed the process. If the parameter is zero (or negative), then snapshots are processed sequentially; otherwise parallelization is active, up to a maximum number of processes given by this parameter. By default, this parameter takes the value of **CONFIG\['DEFAULT'\]['parallelcores']**. 

You must run **_Step 0_** first, to load the configuration, before running this step. 

The parameters for this step cover the name of the **\*.csv** relationships file, file locations for the input and the cache, the quarterly start and end dates to process, and whether or not to rebuild the cache.  

In [5]:
# STEP 3 - Build banking system network snapshots
#
# Creates bare network objects for the full banking system at each quarter in a range
import csv2sys as c2s
# Reload the module, just to ensure the notebook kernel has the latest version from disk
reload(c2s)

# Some configuration overrides:
CONFIG['csv2sys']['attributesactive']='20160630_ATTRIBUTES_ACTIVE.csv'
CONFIG['csv2sys']['attributesbranch']='20160630_ATTRIBUTES_BRANCH.csv'
CONFIG['csv2sys']['attributesclosed']='20160630_ATTRIBUTES_INACTIVE.csv'
CONFIG['csv2sys']['relationships']='20160630_RELATIONSHIPS.csv'
#CONFIG['csv2sys']['asofdate0']='2007Q1'
CONFIG['csv2sys']['asofdate1']='2016Q2'

# Now, do the actual work:
c2s.build_sys(CONFIG)

100% (122 of 122) |######################| Elapsed Time: 0:14:51 Time:  0:14:51
N/A% (0 of 122) |                        | Elapsed Time: 0:29:44 ETA:  --:--:--

### Step 4 - Build individual BHC network snapshots
This step creates (and caches) network objects for quarterly snapshots of individual BHCs. Each quarterly BHC network snapshot derives from the NIC **Relationships** table, adding details from the various **Attributes** tables. 

It is prohibitive to cache a snapshot for every BHC and every quarter in the sample (there are thousands of BHCs and scores of quarters, implying millions of BHC-quarter caches).  The parameters **asoflist** and **bhclist** control the composition of the sample. The **asoflist** parameter contains strings of the form _yyyyQn_, where _yyyy_ represents the year and _n_ represents the calendar quarter (1-4).  The **bhclist** parameter contains integers for the RSSDs (high-holder only) of the BHCs to include. 

The network objects are pickled and cached in the output directory, **outdir**. To speed performance, this step checks whether a cached **\*.pik** file exists before creating a new network object. To override this behavior, ensure that the **clearcache** parameter is set to _True_; this will force the deletion (and recreation) of cached network object for the selected quarters. 

The **parallel** parameter toggles whether to exploit multiple CPU cores to speed the process. If the parameter is zero (or negative), then snapshots are processed sequentially; otherwise parallelization is active, up to a maximum number of processes given by this parameter. By default, this parameter takes the value of **CONFIG\['DEFAULT'\]['parallelcores']**. 

You must run **_Step 0_** first, to load the configuration, before running this step. 

The parameters for this step cover the name of the **\*.csv** relationships file, file locations for the input and the cache, the quarterly start and end dates to process, and whether or not to rebuild the cache.  

In [6]:
# STEP 4 - Build individual BHC network snapshots
#
# Creates (and caches) network objects for limited sets of dates and BHCs
import csv2sys as c2s
import sys2bhc as s2b
# Reload the modules, just to ensure the notebook kernel has the latest version from disk
reload(c2s)
reload(s2b)

# Some configuration overrides:
CONFIG['sys2bhc']['relationships']='20160630_RELATIONSHIPS.csv'
CONFIG['sys2bhc']['attributesactive']='20160630_ATTRIBUTES_ACTIVE.csv'
CONFIG['sys2bhc']['attributesbranch']='20160630_ATTRIBUTES_BRANCH.csv'
CONFIG['sys2bhc']['attributesclosed']='20160630_ATTRIBUTES_INACTIVE.csv'
CONFIG['sys2bhc']['asoflist']='["2006Q4","2008Q3","2008Q4","2010Q4"]'
CONFIG['sys2bhc']['bhclist']='[1073551,1120754]'         # RSSDs: 1073551=Wachovia; 1120754=Wells Fargo
CONFIG['sys2bhc']['parallel']='0'

# Now, do the actual work:
s2b.make_bhcs(CONFIG)

100% (4 of 4) |##########################| Elapsed Time: 0:01:21 Time:  0:01:21
N/A% (0 of 122) |                        | Elapsed Time: 0:41:27 ETA:  --:--:--

### Step 5 - Build outputs

In [7]:
# STEP 5 - Build outputs
#
# Creates (and caches) network objects for limited sets of dates and BHCs
import csv2sys as c2s
import sys2bhc as s2b
import bhc2out as b2o
# Reload the modules, just to ensure the notebook kernel has the latest version from disk
reload(c2s)
reload(s2b)
reload(b2o)

# Some configuration overrides:
#CONFIG['bhc2out']['parallel']='0'
#CONFIG['bhc2out']['make_panel']='False'
#CONFIG['DEFAULT']['veryverbose']='True'

# Now, do the actual work:
UTIL.print_config(CONFIG, 'bhc2out')
#b2o.make_panel(CONFIG)
b2o.make_wachwells_comparison(CONFIG)


BHC-Quarter pairs: 100% |######################################| ETA:  00:00:00

,rssd,asofdate,Bas_Vertex_count,Bas_Edge_count,Bas_Cycle_rank,Bas_Num_CComp,Ent_Qfull_B1,Ent_Qhetr_B1,Ent_Qfcon_B1,Ent_Qhcon_B1,...,Ent_DjHom_M,Ent_Nlabl,Geo_Qfull_B1,Geo_Qhetr_B1,Geo_Qfcon_B1,Geo_Qhcon_B1,Geo_edgcn_B1,Geo_DjHom_B1,Geo_DjHom_M,Geo_Nlabl
1073551_20061231,1073551,20061231,1384,1673,290,1,1658,454,17,14,...,249,16,1614,729,47,28,18,217,716,60
1073551_20080930,1073551,20080930,1034,1438,405,1,1424,674,18,14,...,296,15,1386,295,48,32,17,366,309,53
1120754_20061231,1120754,20061231,603,641,39,1,628,216,16,12,...,200,14,588,338,89,70,19,7,360,54
1120754_20080930,1120754,20080930,607,637,31,1,624,223,17,13,...,214,14,584,332,85,67,15,6,361,54
1120754_20081231,1120754,20081231,1692,2137,446,1,2122,982,27,22,...,571,16,2068,763,160,134,33,380,767,70
1120754_20101231,1120754,20101231,1521,1999,479,1,1985,974,20,16,...,520,15,1932,1121,137,113,32,5,715,68
